In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['font.size'] = 10
import seaborn as sns

모델 평가

오버피팅(overfitting)과 언더피팅(underfitting)  

오버피팅은 특정 데이터 셋에 과도하게 적합된 것을 의미한다. 오버피팅이 발생하는 경우, 얼핏 정확도가 높아 보이지만 특정 데이터 셋에만 적합되어 알려지지 않은 데이터에 대한 예측력은 낮아지게 된다. 언더피팅은 데이터 셋에 적합이 잘되지 않은, 즉 과소적합된 것을 의미한다.  
머신러닝을 통해 모델을 학습하는 이유는 데이터의 종류와 상관없이 일반화할 수 있는 모델을 생성하는 것이다. 주어진 데이터 셋에 대해 오버피팅이나 언더피팅이 발생한다면 새로운 데이터에 적용할 수 있는 좋은 모델이라고 말하기 어렵다.  
오버피팅된 모델을 새로운 데이터셋에 적용한다면 학습 데이터 셋과는 큰 오차를 보인다. 반대로 언더피팅은 학습 데이터 셋과 테스트 데이터 셋 모두 큰 오차를 보인다.

편향-분산 트레이드오프(bias-variance tradeoff)  

편향 분산 트레이드오프란, 편향이 낮을수록 분산은 커지고, 반대로 편향이 높을수록 분산이 작아지는 경향이 있다는 것을 의미한다.  
분산이 높은 현상은 주로 복잡한 모델에 나타나고 모델이 복잡하다는 말은 오버피팅이 발생할 가능성이 높다는 뜻이다. 즉, 복잡한 모델일수록 오버피팅이 발생할 가능성이 높으며, 이는 분산이 커진다는 것을 의미한다.  
반대로 편향이 큰 현상은 주로 간단한 모델일 때 나타나는데 모델이 간단하다는 말은 언더피팅이 발생할 가능성이 높다는 뜻이다. 간단한 모델일수록 언더피팅이 발생할 가능성이 높으며, 이에 따라 편향이 커질 수 있다.

크로스 벨리데이션(cross validation, 교차 검증)  

모델을 생성한 후 실제 데이터에 적용해 보고 성능을 평가해야 하는데, 데이터 셋 전체를 학습에 사용하면 새롭게 적용할 데이터가 없어서 문제가 발생되기 때문에 전체 데이터를 학습 데이터와 테스트 데이터로 분할해서 사용한다. 학습 데이터는 학습하는 데 사용되고, 테스트 데이터는 학습 시에는 사용되지 않고 모델의 성능을 평가할 때 사용한다.  
머신러닝 알고리즘을 적용할 때 다양한 하이퍼파라미터에 대해 여러 가지 모델 후보군을 생성하고 평가한 후 최종 모델을 선택하게 된다. 이때, 파라미터는 모델 내부에서 데이터에 의해 추정되는 값이고, 하이퍼파라미터는 사용자가 직접 정하는 값이다.  
하이퍼파라미터를 결정하는 과정에서 학습 데이터와 테스트 데이터만 존재한다면 테스트 데이터에 의해 최종 모델의 파라미터가 결정된다. 즉, 모델의 하이퍼파라미터가 테스트 데이터에 의존한다는 뜻이다.  
이 문제를 해결하기 위해 학습 데이터의 일부를 검증 데이터로 사용한다. 즉, 학습 데이터는 파라미터를 구하는데 사용하고, 검중 데이터는 하이퍼파라미터를 정하는데 사용한다.  
주어진 데이터셋에 대해서 학습 데이터, 검증 데이터, 테스트 데이터로 분할할 수 있는 다양한 조합 방법이 존재한다. 이처럼 다양한 조합을 통해 모델의 성능을 검증하는 것을 크로스-밸리데이션이라고 한다.

In [14]:
from sklearn import datasets # 사이킷런이 제공하는 데이터 셋을 사용하기 위해 import 한다.
from sklearn.model_selection import train_test_split # 학습 데이터와 테스트 데이터를 나누기 위해 import 한다.
from sklearn.preprocessing import StandardScaler # 표준화 스케일링을 하기 위해 import 한다.
from sklearn.linear_model import LinearRegression # 사이킷런이 제공하는 선형 회귀 모델을 사용하기 위해 import 한다.
from sklearn.neighbors import KNeighborsClassifier # 사이킷런이 제공하는 최근접 이웃 모델을 사용하기 위해 import 한다.
from sklearn.pipeline import Pipeline # 파이프라인을 사용하기 위해 import 한다.

from sklearn.metrics import mean_squared_error # 평균 제곱 오차(MSE)를 계산하기 위해 import 한다.
from sklearn.metrics import mean_absolute_error # 평균 절대값 오차(MAE)를 계산하기 위해 import 한다.
from sklearn.metrics import accuracy_score # 정확도를 계산하기 위해 import 한다.
from sklearn.metrics import r2_score # R 제곱값을 계산하기 위해 import 한다.
from sklearn.metrics import silhouette_score # 실루엣 스코어를 계산하기 위해 import 한다.

In [3]:
# 사이킷런이 제공하는 보스턴 집값 데이터 셋에서 피쳐와 레이블 데이터 읽기
raw_data = datasets.load_boston()
xData = raw_data.data # 데이터
yData = raw_data.target # 데이터에 따른 레이블
print(xData.shape, yData.shape)

(506, 13) (506,)


파이프라인  

머신러닝 과정에서 파이프라인을 사용하면 데이터 전처리와 학습 모델을 연결해 코드를 간결화 할 수 있다.

파이프라인을 사용하지 않은 코드

In [4]:
# 학습 데이터와 테스트 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(xData, yData, random_state=7)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
# 표준화 스케일링 적용
std_scale = StandardScaler()
x_train = std_scale.fit_transform(x_train) # 학습 데이터 스케일링
x_test = std_scale.transform(x_test) # 테스트 데이터 스케일링
# 모델 생성 후 학습
clf = LinearRegression() # 선형 회귀 모델을 만든다.
clf.fit(x_train, y_train) # 스케일링된 학습 데이터와 학습 데이터에 따른 레이블로 학습시킨다.
# 모델 예측
predict = clf.predict(x_test) # 스케일링된 테스트 데이터로 학습된 모델을 예측한다.
# 모델 평가
# 테스트 데이터의 레이블과 테스트 데이터로 예측한 값의 평균 제곱 오차를 계산한다.
print(mean_squared_error(y_test, predict))
# 테스트 데이터의 레이블과 테스트 데이터로 예측한 값의 평균 절대값 오차를 계산한다.
print(mean_absolute_error(y_test, predict))

(379, 13) (127, 13) (379,) (127,)
29.51513779019758
3.5678438946275453


파이프라인을 사용하는 코드

In [5]:
# 학습 데이터와 테스트 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(xData, yData, random_state=7)

# 파이프라인 => 표준화 스케일링(전처리) + 학습 모델 객체 생성
pipeline = Pipeline(
    [
        ('scaler', StandardScaler()), # 표준화 스케일링 적용
        ('linear_regression', LinearRegression()) # 선형 회귀 모델 적용
    ]
)

# 파이프라인에서 스케일러와 모델 생성 후 학습
pipeline.fit(x_train, y_train)
# 모델 예측
predict = pipeline.predict(x_test)
# 모델 평가
print(mean_squared_error(y_test, predict))
print(mean_absolute_error(y_test, predict))

29.51513779019758
3.5678438946275453


그리드 서치(Grid Search)

머신러닝 과정에서 관심잇는 매개변수들을 대상으로 학습 가능하도록 만드는 방식을 말한다.

k-최근접 이웃 알고리즘 사용시 1부터 10 사이의 k값 후보 중 가장 높은 성능을 보이는 k값 정하기

In [8]:
# 사이킷런이 제공하는 붓꽃 셋에서 피쳐와 레이블 데이터 읽기
raw_data = datasets.load_iris()
xData = raw_data.data
yData = raw_data.target
print(xData.shape, yData.shape)

(150, 4) (150,)


In [17]:
# 학습 데이터와 테스트 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(xData, yData, random_state=0)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
# 표준화 스케일링 적용
std_scale = StandardScaler()
x_train = std_scale.fit_transform(x_train)
x_test = std_scale.transform(x_test)

best_accuracy = 0 # 최고 정확도를 기억할 변수를 선언하고 0으로 초기화 한다.
# 1부터 10 사이의 k값 후보 중 가장 높은 성능을 보이는 k값을 찾는다.
for k in range(1, 11):
    # n_neighbors 인수로 k값을 넘겨서 최근접 이웃 모델을 만든다.
    clf_knn = KNeighborsClassifier(n_neighbors=k)
    # 학습 데이터와 학습 데이터에 따른 레이블을 넘겨서 최근접 이웃 모델을 학습시킨다.
    clf_knn.fit(x_train, y_train)
    # 학습된 모델에 테스트 데이터를 넘겨서 예측한다.
    predict = clf_knn.predict(x_test)
    # 테스트 데이터의 레이블과 예측값을 넘겨서 정확도를 계산한다.
    accuracy = accuracy_score(y_test, predict)
    print('k: {:2d}, accuracy: {:6.3f}'.format(k, accuracy))
    
    # 가장 정확도가 높은 k를 계산한다.
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        final_k = k
    # ===== if
# ===== for
print('best k: {:2d}, accuracy: {:6.3f}'.format(final_k, accuracy))

(112, 4) (38, 4) (112,) (38,)
k:  1, accuracy:  0.921
k:  2, accuracy:  0.947
k:  3, accuracy:  0.974
k:  4, accuracy:  0.974
k:  5, accuracy:  0.974
k:  6, accuracy:  0.974
k:  7, accuracy:  0.974
k:  8, accuracy:  0.974
k:  9, accuracy:  0.974
k: 10, accuracy:  0.974
best k:  3, accuracy:  0.974
